In [1]:
# IMPORTS
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import playercareerstats, playergamelog, teamgamelog
from nba_api.live.nba.endpoints import scoreboard, playbyplay, boxscore
from datetime import datetime, timezone
from dateutil import parser
import pandas as pd
from pprint import pprint

In [2]:
# Specify the player name you're interested in
target_player_name = input("Enter the player name: ")

Enter the player name:  Tyrese Maxey


In [3]:
# PLAYER CAREER STATS

# Specify the player name you're interested in
#target_player_name = 'Joel Embiid'  # Replace this with the desired player name

# Get the list of all NBA players
nba_players = players.get_players()

# Find the player with the specified name
target_player = next(player for player in nba_players if player['full_name'] == target_player_name)
# {'id': 203954, 'full_name': 'Joel Embiid', 'first_name': 'Joel', 'last_name': 'Embiid', 'is_active': True}

# Extract the relevant data for the player's career
career_stats = playercareerstats.PlayerCareerStats(player_id=target_player['id'])
career_per_season_data = career_stats.get_data_frames()[0]
    
# Create a DataFrame for the player's career stats
player_career_stats_df = pd.DataFrame(career_per_season_data)
    
# Create a DataFrame with player information
nba_players_df = pd.DataFrame(nba_players)
    
# Merge the two DataFrames based on 'PLAYER_ID'
player_career_df = pd.merge(player_career_stats_df, nba_players_df[['id', 'full_name']], left_on='PLAYER_ID', right_on='id', how='left')
    
# Drop the redundant 'id' column
player_career_df = player_career_df.drop(columns=['id'])
    
# Reorder the columns to make 'full_name' the second column and rename  to 'FULL_NAME'
player_career_df = player_career_df[['PLAYER_ID', 'full_name'] + [col for col in player_career_df.columns if col not in ['PLAYER_ID', 'full_name']]].rename(columns={'full_name': 'FULL_NAME'})

# Display the merged DataFrame
player_career_df

,PLAYER_ID,FULL_NAME,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1630178,Tyrese Maxey,2020-21,00,1610612755,PHI,20.0,61,8,936.0,...,0.871,11,93,104,120,26,13,41,79,488
1,1630178,Tyrese Maxey,2021-22,00,1610612755,PHI,21.0,75,74,2650.0,...,0.866,26,214,240,321,55,32,88,156,1311
2,1630178,Tyrese Maxey,2022-23,00,1610612755,PHI,22.0,60,41,2016.0,...,0.845,23,153,176,212,49,8,80,132,1218
3,1630178,Tyrese Maxey,2023-24,00,1610612755,PHI,23.0,11,11,418.0,...,0.945,8,48,56,74,9,10,16,22,304


In [4]:
# PLAYERS GAME LOG 

# Player will be same as above
game_log = playergamelog.PlayerGameLog(player_id=target_player['id']) 
game_log_data = game_log.get_data_frames()[0]

# Create a DataFrame for the player's career stats
game_log_df = pd.DataFrame(game_log_data)

# Merge the two DataFrames based on 'PLAYER_ID'
game_log_df = pd.merge(game_log_df, nba_players_df[['id', 'full_name']], left_on='Player_ID', right_on='id', how='left')

# Drop the redundant 'id' column
game_log_df = game_log_df.drop(columns=['id'])

# Reorder the columns to make 'full_name' the second column and rename  to 'FULL_NAME'
game_log_df = game_log_df[['Player_ID', 'full_name'] + [col for col in game_log_df.columns if col not in ['Player_ID', 'full_name']]].rename(columns={'full_name': 'FULL_NAME'})

game_log_df

,Player_ID,FULL_NAME,SEASON_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,1630178,Tyrese Maxey,22023,0022300194,"NOV 15, 2023",PHI vs. BOS,L,39,6,15,...,3,3,4,0,0,2,3,20,9,1
1,1630178,Tyrese Maxey,22023,0022300019,"NOV 14, 2023",PHI vs. IND,L,38,9,23,...,4,4,5,0,1,4,2,27,-6,1
2,1630178,Tyrese Maxey,22023,0022300180,"NOV 12, 2023",PHI vs. IND,W,39,20,32,...,4,7,5,0,3,2,2,50,17,1
3,1630178,Tyrese Maxey,22023,0022300008,"NOV 10, 2023",PHI @ DET,W,39,9,19,...,6,6,11,3,1,0,3,29,17,1
4,1630178,Tyrese Maxey,22023,0022300159,"NOV 08, 2023",PHI vs. BOS,W,39,11,27,...,8,9,5,1,2,1,4,25,-3,1
5,1630178,Tyrese Maxey,22023,0022300148,"NOV 06, 2023",PHI vs. WAS,W,32,9,16,...,1,1,11,1,1,0,2,22,9,1
6,1630178,Tyrese Maxey,22023,0022300132,"NOV 04, 2023",PHI vs. PHX,W,36,9,18,...,3,5,10,0,1,3,0,22,25,1
7,1630178,Tyrese Maxey,22023,0022300128,"NOV 02, 2023",PHI vs. TOR,W,40,7,16,...,1,1,4,2,0,1,1,18,6,1
8,1630178,Tyrese Maxey,22023,0022300098,"OCT 29, 2023",PHI vs. POR,W,35,6,14,...,9,10,4,0,0,2,1,26,19,1
9,1630178,Tyrese Maxey,22023,0022300092,"OCT 28, 2023",PHI @ TOR,W,41,12,20,...,5,6,7,0,1,1,1,34,10,1


In [59]:
# LIST OF ALL PLAYERS
nba_players = players.get_players() # Get the list of all NBA players

# Create a DataFrame with player information
nba_players_df = pd.DataFrame(nba_players)

# Print nba list
print("Number of players fetched: {}".format(len(nba_players)))
nba_players_df.head()

Number of players fetched: 4900


,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


In [56]:
# FIND SPECIFIC PLAYER
target_player_name = 'LeBron James'  # Replace this with the desired player name

# Search for the player with the specified name
target_player_df = nba_players_df.query(f'full_name == "{target_player_name}"')
target_player_df

,id,full_name,first_name,last_name,is_active
2111,2544,LeBron James,LeBron,James,True


In [61]:
# LIST OF NBA TEAMS

# Get the list of all NBA teams
nba_teams = teams.get_teams()

# Create team df
teams_df = pd.DataFrame(nba_teams)

# Print team data
teams_df

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [113]:
# EXTRACT SPECIFIC TEAMS
sixers = [team for team in nba_teams if team["full_name"] == "Philadelphia 76ers"][0]
spurs = [team for team in nba_teams if team["full_name"] == "San Antonio Spurs"][0]

sixers

{'id': 1610612755,
 'full_name': 'Philadelphia 76ers',
 'abbreviation': 'PHI',
 'nickname': '76ers',
 'city': 'Philadelphia',
 'state': 'Pennsylvania',
 'year_founded': 1949}

In [9]:
# TODAYS GAMES
board = scoreboard.ScoreBoard() # Get the scoreboard for today's games
games_data = board.games.get_dict()

# Create a list to store game information
game_list = []

# Format the game information and append it to the list
for game in games_data:
    gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
    game_info = {
        "GameId": game['gameId'],
        "AwayTeam": game['awayTeam']['teamTricode'],
        "HomeTeam": game['homeTeam']['teamTricode'],
        "GameTimeLTZ": gameTimeLTZ
    }
    game_list.append(game_info)

# Convert the list of dictionaries to a DataFrame
games_df = pd.DataFrame(game_list)
games_df


,GameId,AwayTeam,HomeTeam,GameTimeLTZ
0,0022300225,WAS,CHA,2023-11-22 19:00:00-05:00
1,0022300226,DEN,ORL,2023-11-22 19:00:00-05:00
2,0022300227,BKN,ATL,2023-11-22 19:30:00-05:00
3,0022300228,MIL,BOS,2023-11-22 19:30:00-05:00
4,0022300229,MIA,CLE,2023-11-22 19:30:00-05:00
5,0022300230,TOR,IND,2023-11-22 19:30:00-05:00
6,0022300231,MEM,HOU,2023-11-22 20:00:00-05:00
7,0022300232,PHI,MIN,2023-11-22 20:00:00-05:00
8,0022300233,SAC,NOP,2023-11-22 20:00:00-05:00
9,0022300234,CHI,OKC,2023-11-22 20:00:00-05:00


In [21]:
def is_team_home_or_away(team_abbreviation, game_row):
    # Check if the team abbreviation is in the 'HomeTeam' column
    if team_abbreviation == game_row['HomeTeam']:
        return True
    # Check if the team abbreviation is in the 'AwayTeam' column
    elif team_abbreviation == game_row['AwayTeam']:
        return False
    # If the team abbreviation is not found in either column
    else:
        return None

# Example usage:
target_team_abbreviation = 'UTA'
for index, game_row in games_df.iterrows():
    is_home = is_team_home_or_away(target_team_abbreviation, game_row)
    if is_home is not None:
        print(f'Home game: {is_home}')

Home game: False


In [117]:
# GAME INFO
game_id = '0022300194'  # Replace with your desired game ID
box = boxscore.BoxScore(game_id)

# Get game information
game_info = box.game.get_dict()
date = game_info['gameTimeLocal']
away_team = game_info['awayTeam']['teamName']
home_team = game_info['homeTeam']['teamName']

print(f"Game ID: {game_id}")
print(f"Date: {date}")
print(f"Away Team: {away_team}")
print(f"Home Team: {home_team}")

Game ID: 0022300194
Date: 2023-11-15T19:30:00-05:00
Away Team: Celtics
Home Team: 76ers


In [120]:
# BOX SCORES  
box = boxscore.BoxScore('0022300194') # Choose Game

# Note: home_team & away_team have the identicial data structure.
players_data = box.home_team.get_dict()['players']

# Create a list of dictionaries
data_list = []
for player in players_data:
    three_pointers_attempted = player['statistics']['threePointersAttempted']
    three_pointers_made = player['statistics']['threePointersMade']

    # Calculate shooting percentage, handle the case where attempted is zero to avoid division by zero
    shooting_percentage = f"{three_pointers_made}/{three_pointers_attempted}" if three_pointers_attempted > 0 else "0/0"

    player_dict = {
        'player_id': player['personId'],
        'name': player['name'],
        'points': player['statistics']['points'],
        'rebounds': player['statistics']['reboundsTotal'],
        'assists': player['statistics']['assists'],
        '3pt %': shooting_percentage
    }
    data_list.append(player_dict)

# Create DataFrame
boxscore_df = pd.DataFrame(data_list)

# Display the DataFrame
boxscore_df

,player_id,name,points,rebounds,assists,3pt %
0,203496,Robert Covington,10,3,0,0/3
1,202699,Tobias Harris,12,3,2,1/1
2,203954,Joel Embiid,10,5,3,0/1
3,1629001,De'Anthony Melton,6,2,1,2/2
4,1630178,Tyrese Maxey,10,3,4,2/4
5,202694,Marcus Morris Sr.,0,0,0,0/1
6,201976,Patrick Beverley,0,1,1,0/1
7,1627863,Danuel House Jr.,0,0,0,0/1
8,1627788,Furkan Korkmaz,0,0,0,0/1
9,1630531,Jaden Springer,7,2,0,1/1
